In [1]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [2]:
import os

# 没有GPU注释以下代码

# os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# # 获取物理gpu, cpu对象
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')

# # 设置当前程序的　物理可见设备范围
# tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

# #设置仅在需要时申请：
# # for gpu in gpus:
# #     tf.config.experimental.set_memory_growth(gpu, True)

# # 设置在物理gpu上设置虚拟gpu，并用来限制gpu内存使用
# tf.config.experimental.set_virtual_device_configuration(
#     gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024), tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

# # 获取当前虚拟gpu对象
# logical_gpus = tf.config.experimental.list_logical_devices('GPU')

# print('物理gpu个数: ', len(gpus))
# print('总的gpu个数，其中包括括虚拟Logical gpus: ', len(logical_gpus))

In [3]:
# 获取数据
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 定义预处理函数
def data_scale(x, y):
    x = tf.cast(x, tf.float32)
    # 由于还没有学卷积，所以需要把图片数据展开,28*28
    x = x / 255
    x = tf.reshape(x,(784, -1))    # (78,1)
    x = tf.squeeze(x)    # 降一个维度
    return x, y

train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_scale).cache().batch(64)
test_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_scale).cache().batch(64)
# next(iter(train_db))  # (shape=(64, 784))

In [4]:
# 创建模型
# 由于还没有学卷积，所以需要把图片数据展开，展开图像的长度为 28*28
num_inputs = 784
# 分类类别数为: 10 分类
num_outputs = 10
# 构建两个隐藏层
W1 = tf.Variable(tf.random.normal(shape=(num_inputs, 128),mean=0, stddev=0.1, dtype=tf.float32))
B1 = tf.Variable(tf.random.normal(shape=(1,128), mean=0.5, stddev=0.1, dtype=tf.float32))

W2 = tf.Variable(tf.random.normal(shape=(128, 64),mean=0, stddev=0.1, dtype=tf.float32))
B2 = tf.Variable(tf.random.normal(shape=(1,64), mean=0.5, stddev=0.1, dtype=tf.float32))

W3 = tf.Variable(tf.random.normal(shape=(64, 10),mean=0, stddev=0.1, dtype=tf.float32))
B3 = tf.Variable(tf.random.normal(shape=(1,10), mean=0.5, stddev=0.1, dtype=tf.float32))
def hidden_layer_net(X,W1,B1,W2,B2,W3,B3):
    h1 = tf.nn.relu(tf.matmul(X, W1) + B1)
    h2 = tf.nn.relu(tf.matmul(h1, W2) + B2)
    out_put = tf.math.softmax(tf.matmul(h2, W3) + B3)
    return out_put

In [5]:
# 定义损失函数
def cross_entropy(y_hat, y):
    y = tf.cast(tf.reshape(y, shape=[-1, 1]),dtype=tf.int32)
    y = tf.one_hot(y, depth=y_hat.shape[-1])    
    y = tf.cast(tf.reshape(y, shape=[-1, y_hat.shape[-1]]),dtype=tf.int32)
    return -tf.math.log(tf.boolean_mask(y_hat, y))

In [6]:
# 定义评估函数
def evaluate_accuracy(data_iter, hidden_layer_net):
    acc_sum, n = 0, 0.0
    for x, y in data_iter:
        yhat = hidden_layer_net(x, W1,B1,W2,B2,W3,B3)
        acc_sum += tf.reduce_sum(tf.cast(tf.argmax(yhat, axis=1) == tf.cast(y, dtype=tf.int64), dtype=tf.float32))
        n += y.shape[0]
    return acc_sum / n

In [7]:
# 训练模型
epoches = 20
lr = 1e-3

trainer = tf.keras.optimizers.SGD(lr)

def train_model(trainer=trainer):
    for epoch in range(epoches):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0.0
        for (x_in, y_in) in train_db:
            # 创建上下文管理器，链接需要计算梯度的函数和变量
            with tf.GradientTape() as tape:
                y_out = hidden_layer_net(x_in, W1,B1,W2,B2,W3,B3)
                loss = tf.reduce_sum(cross_entropy(y_out, y_in))
#                 print('loss', loss)
            # 自动观测可训练变量
            #grads = tape.gradient(loss, [W, B])
            grads = tape.gradient(loss, [W1,B1,W2,B2,W3,B3])
            if trainer:
                # 使用优化器自动更新参数
                trainer.apply_gradients(zip(grads, [W1,B1,W2,B2,W3,B3]))
            else:
                # 手动更新参数
                W.assign_sub(grads[0] * lr)
                B.assign_sub(grads[1] * lr)
            # 计算训练集批量平均准确率和累计平均损失,注意这样做主要是为了方便计算，更合理应该计算全局损失和准确率,但是又需要重新计算，耗时较长
            train_acc_sum += tf.reduce_sum(tf.cast(tf.argmax(y_out, axis=1) == tf.cast(y_in, dtype=tf.int64), dtype=tf.float32))
#             print(loss)
            train_l_sum += loss
            # 样本个数：
            n += y_in.shape[0]
        test_acc = evaluate_accuracy(test_db, hidden_layer_net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_model(trainer=trainer)

Instructions for updating:
Use tf.identity instead.
epoch 1, loss 0.5975, train acc 0.785, test acc 0.837
epoch 2, loss 0.4303, train acc 0.844, test acc 0.857
epoch 3, loss 0.3884, train acc 0.858, test acc 0.869
epoch 4, loss 0.3614, train acc 0.868, test acc 0.876
epoch 5, loss 0.3413, train acc 0.874, test acc 0.882
epoch 6, loss 0.3253, train acc 0.880, test acc 0.887
epoch 7, loss 0.3118, train acc 0.885, test acc 0.890
epoch 8, loss 0.3003, train acc 0.889, test acc 0.894
epoch 9, loss 0.2897, train acc 0.893, test acc 0.897
epoch 10, loss 0.2800, train acc 0.897, test acc 0.900
epoch 11, loss 0.2717, train acc 0.900, test acc 0.903
epoch 12, loss 0.2635, train acc 0.903, test acc 0.905
epoch 13, loss 0.2558, train acc 0.906, test acc 0.907
epoch 14, loss 0.2486, train acc 0.908, test acc 0.909
epoch 15, loss 0.2419, train acc 0.911, test acc 0.912
epoch 16, loss 0.2352, train acc 0.914, test acc 0.913
epoch 17, loss 0.2290, train acc 0.916, test acc 0.915
epoch 18, loss 0.2226,